In [1]:
#!pip install pandas-profiling==3.3.0
#!pip install xlsxwriter
!pip install pandas==2.0.3

In [2]:
import pandas as pd
import numpy as np
import babel.dates

In [3]:
df_us_all = pd.read_excel(r'C:\Users\Kkk\OneDrive\Documentos\DNC\Projetos\SG Global - Power BI\DRE US - DNC .xlsx', sheet_name="Base")
df_br_all = pd.read_excel(r'C:\Users\Kkk\OneDrive\Documentos\DNC\Projetos\SG Global - Power BI\DRE BR - DNC.xlsx', sheet_name="Base")
df_pc_all = pd.read_excel(r'C:\Users\Kkk\OneDrive\Documentos\DNC\Projetos\SG Global - Power BI\DRE US - DNC .xlsx', sheet_name="Base")

In [4]:
df_pc = df_pc_all[['Conta', 'Categoria']]
df_pc = df_pc.rename(columns={'Conta': 'Categoria_Num'})
df_pc['Total'] = 0

In [5]:
#identificar as bases
df_us_all = df_us_all.assign(Origem='US')
df_br_all = df_br_all.assign(Origem='BR')

In [6]:
#remover espaços na frente do nome das colunas
df_us_all.columns = df_us_all.columns.str.strip()
df_br_all.columns = df_br_all.columns.str.strip()

In [7]:
df_us = df_us_all[['Origem','Data de Competência', 'Categoria','Descrição', 'Cliente / Fornecedor','Valor recebido', 'Valor pago','Conta','Centro de custo', 'Empresa']]
df_br = df_br_all[['Origem','Data de Competência', 'Categoria','Descrição', 'Cliente / Fornecedor','Valor recebido', 'Valor pago','Conta','Centro de custo', 'Empresa']]

In [8]:
#excluir a primeira linha da base / cabeçalho
df_br = df_br.iloc[1:]

In [9]:
df = pd.concat([df_us,df_br])

In [10]:
df['Mês'] = df['Data de Competência'].dt.strftime('%B')
df['Ano'] = df['Data de Competência'].dt.year

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51024 entries, 0 to 19968
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Origem                51024 non-null  object        
 1   Data de Competência   50022 non-null  datetime64[ns]
 2   Categoria             50022 non-null  object        
 3   Descrição             50022 non-null  object        
 4   Cliente / Fornecedor  50022 non-null  object        
 5   Valor recebido        28737 non-null  float64       
 6   Valor pago            13630 non-null  float64       
 7   Conta                 49834 non-null  object        
 8   Centro de custo       39187 non-null  object        
 9   Empresa               50022 non-null  object        
 10  Mês                   50022 non-null  object        
 11  Ano                   50022 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(8)
memory usage: 5.1+ MB


In [12]:
df.head()

,Origem,Data de Competência,Categoria,Descrição,Cliente / Fornecedor,Valor recebido,Valor pago,Conta,Centro de custo,Empresa,Mês,Ano
0,US,2023-01-01,01.02.01.01.01 - EB1 - Honorários de Imigração,DESCRIÇÃO 956,CLIENTE 191,1300.0,NaN,14.01.03 - Accounts Receivable (A/R),Comercial,SG LAW,January,2023.0
1,US,2023-01-01,01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,DESCRIÇÃO 640,CLIENTE 164,1300.0,NaN,14.01.03 - Accounts Receivable (A/R),Comercial,SG LAW,January,2023.0
2,US,2023-01-01,01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,DESCRIÇÃO 988,CLIENTE 76,1300.0,NaN,14.01.03 - Accounts Receivable (A/R),Comercial,SG LAW,January,2023.0
3,US,2023-01-01,01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,DESCRIÇÃO 940,CLIENTE 77,1300.0,NaN,14.01.03 - Accounts Receivable (A/R),Comercial,SG LAW,January,2023.0
4,US,2023-01-01,01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,DESCRIÇÃO 992,CLIENTE 13,1300.0,NaN,14.01.03 - Accounts Receivable (A/R),Comercial,SG LAW,January,2023.0


In [13]:
# criando chaves extrangeiras onde ausentes (identificadas no excel)
df.loc[df['Categoria'] == 'Transferência de Entrada', 'Categoria'] = '01' + df.loc[df['Categoria'] == 'Transferência de Entrada', 'Categoria']
df.loc[df['Categoria'] == 'Transferência de Saída', 'Categoria'] = '02' + df.loc[df['Categoria'] == 'Transferência de Saída', 'Categoria']
df.loc[df['Categoria'] == 'Saldo Inicial', 'Categoria'] = '00 - ' + df.loc[df['Categoria'] == 'Saldo Inicial', 'Categoria']
df.loc[df['Categoria'] == 'Descontos', 'Categoria'] = '99 - ' + df.loc[df['Categoria'] == 'Descontos', 'Categoria']

In [14]:
#separar categóricas de númericas e excluir coluna desnecessária no momento
df[['Categoria_Num', 'Categoria']] = df['Categoria'].str.split('-', n=1, expand=True)
df[['Conta_Num', 'Conta']] = df['Conta'].str.split('-', n=1, expand=True)
df['Categoria_Num'] = df['Categoria_Num'].str.replace('.', '')

In [15]:
#df.drop(columns=['Conta_Num'], inplace=True)

In [16]:
#reordenar
df = df [['Origem','Data de Competência','Mês','Ano', 'Categoria_Num','Categoria','Descrição','Cliente / Fornecedor','Valor recebido','Valor pago','Centro de custo','Conta', 'Empresa']]

In [17]:
#dropando os nulos
df.dropna(how='all', inplace=True)

In [18]:
df.isnull().sum()

Origem                      0
Data de Competência      1002
Mês                      1002
Ano                      1002
Categoria_Num            1002
Categoria                2265
Descrição                1002
Cliente / Fornecedor     1002
Valor recebido          22287
Valor pago              37394
Centro de custo         11837
Conta                   28656
Empresa                  1002
dtype: int64

In [19]:
df.head()

,Origem,Data de Competência,Mês,Ano,Categoria_Num,Categoria,Descrição,Cliente / Fornecedor,Valor recebido,Valor pago,Centro de custo,Conta,Empresa
0,US,2023-01-01,January,2023.0,0102010101,EB1 - Honorários de Imigração,DESCRIÇÃO 956,CLIENTE 191,1300.0,NaN,Comercial,Accounts Receivable (A/R),SG LAW
1,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - Honorários de Imigração,DESCRIÇÃO 640,CLIENTE 164,1300.0,NaN,Comercial,Accounts Receivable (A/R),SG LAW
2,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - Honorários de Imigração,DESCRIÇÃO 988,CLIENTE 76,1300.0,NaN,Comercial,Accounts Receivable (A/R),SG LAW
3,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - Honorários de Imigração,DESCRIÇÃO 940,CLIENTE 77,1300.0,NaN,Comercial,Accounts Receivable (A/R),SG LAW
4,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - Honorários de Imigração,DESCRIÇÃO 992,CLIENTE 13,1300.0,NaN,Comercial,Accounts Receivable (A/R),SG LAW


In [20]:
#removendo espaç
df['Categoria_Num'] = df['Categoria_Num'].str.strip()
df['Conta'] = df['Conta'].str.strip()

In [21]:
#preencher os valores nulos
df_cat = ['Centro de custo', 'Descrição', 'Categoria', 'Cliente / Fornecedor', 'Conta', 'Empresa']
for i in df_cat:
    df[i].fillna('NI', inplace=True)

In [22]:
#preencher os valores nulos
df_num = ['Categoria_Num', 'Valor recebido', 'Valor pago']
for i in df_num:
    df[i].fillna(0, inplace=True)

In [23]:
# padronização caixa alta
for i in df_cat:
    df[i] = df[i].str.upper()

In [24]:
df['Total'] = df['Valor recebido'] - df['Valor pago']
df['Tipo'] = df['Total'].apply(lambda x: 'Entrada' if x > 0 else 'Saída')

In [25]:
df.head(10)

,Origem,Data de Competência,Mês,Ano,Categoria_Num,Categoria,Descrição,Cliente / Fornecedor,Valor recebido,Valor pago,Centro de custo,Conta,Empresa,Total,Tipo
0,US,2023-01-01,January,2023.0,0102010101,EB1 - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 956,CLIENTE 191,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
1,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 640,CLIENTE 164,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
2,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 988,CLIENTE 76,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
3,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 940,CLIENTE 77,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
4,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 992,CLIENTE 13,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
5,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 810,CLIENTE 124,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
6,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 283,CLIENTE 101,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
7,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 587,CLIENTE 58,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
8,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 850,CLIENTE 8,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada
9,US,2023-01-01,January,2023.0,0102010201,EB2 - NIW - HONORÁRIOS DE IMIGRAÇÃO,DESCRIÇÃO 729,CLIENTE 104,1300.0,0.0,COMERCIAL,ACCOUNTS RECEIVABLE (A/R),SG LAW,1300.0,Entrada


In [26]:
#eita coisa linda de ver
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51024 entries, 0 to 19968
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Origem                51024 non-null  object        
 1   Data de Competência   50022 non-null  datetime64[ns]
 2   Mês                   50022 non-null  object        
 3   Ano                   50022 non-null  float64       
 4   Categoria_Num         51024 non-null  object        
 5   Categoria             51024 non-null  object        
 6   Descrição             51024 non-null  object        
 7   Cliente / Fornecedor  51024 non-null  object        
 8   Valor recebido        51024 non-null  float64       
 9   Valor pago            51024 non-null  float64       
 10  Centro de custo       51024 non-null  object        
 11  Conta                 51024 non-null  object        
 12  Empresa               51024 non-null  object        
 13  Total                

In [27]:
#ajustar o tipo  (não vou alterar os valores monetarios em float devido a inconsistência na base, foi sugestionado a empresa o ajuste  )
df['Categoria_Num'] = df['Categoria_Num'].astype(str)
df['Data de Competência'] = pd.to_datetime(df['Data de Competência']).dt.date

In [28]:
  #df.to_csv('Base.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [29]:
#df.to_excel('Base.xlsx', index=False)

In [30]:
df_new = pd.DataFrame(df, )

In [31]:
# tirar os pontos da nossa chave extrangeira nee
df_pc.dropna(how='all', inplace=True)
df['Categoria_Num'] = df['Categoria_Num'].astype(str)
df_pc['Categoria_Num'] = df_pc['Categoria_Num'].str.replace('.', '')

In [32]:
df_pc.head(200)

,Categoria_Num,Categoria,Total
0,140103 - Accounts Receivable (A/R),01.02.01.01.01 - EB1 - Honorários de Imigração,0
1,140103 - Accounts Receivable (A/R),01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,0
2,140103 - Accounts Receivable (A/R),01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,0
3,140103 - Accounts Receivable (A/R),01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,0
4,140103 - Accounts Receivable (A/R),01.02.01.02.01 - EB2 - NIW - Honorários de Imi...,0
...,...,...,...
195,Valley Bank,06.02.05 - Tarifas bancárias,0
196,Valley Bank,06.02.05 - Tarifas bancárias,0
197,Valley Bank,06.02.05 - Tarifas bancárias,0
198,BUS COMPLETE CHK (5762) - 1,06.02.05 - Tarifas bancárias,0


In [33]:
#total_receita_bruta = pd.Series.sum(RECEITA_BRUTA['Total'])
#total_deducao_impostos = pd.Series.sum(DEDUCAO_IMPOSTOS['Total'])

In [34]:
#total_receita_bruta

In [35]:
#total_deducao_impostos

In [36]:
#total_receita_bruta - total_deducao_impostos

In [37]:
#RECEITA_BRUTA = df[(df['Categoria_Num'].str.startswith('01') & (df['Categoria_Num'] != '0101')) | (df['Categoria_Num'] == '140103')].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
#RECEITA_BRUTA.columns = ['Categoria_Num', 'Categoria', 'Total']
#RECEITA_BRUTA['Total'] = RECEITA_BRUTA['Total'].round(2)

# Categorias Princípais

In [38]:
###   (+)
RECEITA_BRUTA = df[(df['Categoria_Num'].str.startswith('01')) | (df['Categoria_Num'] == '140103')].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
RECEITA_BRUTA.columns = ['Categoria_Num', 'Categoria', 'Total']
RECEITA_BRUTA['Total'] = RECEITA_BRUTA['Total'].round(2)


In [39]:
###   (-)
DEDUCAO_IMPOSTOS = df[df['Categoria_Num'].str.startswith('02')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
DEDUCAO_IMPOSTOS.columns = ['Categoria_Num', 'Categoria', 'Total']
DEDUCAO_IMPOSTOS['Total'] = -DEDUCAO_IMPOSTOS['Total'].round(2)

In [40]:
###   (-)
CUSTOS_SERV_VENDIDO = df[df['Categoria_Num'].str.startswith('03')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
CUSTOS_SERV_VENDIDO.columns = ['Categoria_Num', 'Categoria', 'Total']
CUSTOS_SERV_VENDIDO['Total'] = -CUSTOS_SERV_VENDIDO['Total'].round(2)

In [41]:
###   (-)
DESPESAS_OPERACIONAIS = df[df['Categoria_Num'].str.startswith('04')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
DESPESAS_OPERACIONAIS.columns = ['Categoria_Num', 'Categoria', 'Total']
DESPESAS_OPERACIONAIS['Total'] = -DESPESAS_OPERACIONAIS['Total'].round(2)

In [42]:
###   (-)
DEPRECIACOES_AMORTIZACOES = df[df['Categoria_Num'].str.startswith('05')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
DEPRECIACOES_AMORTIZACOES.columns = ['Categoria_Num', 'Categoria', 'Total']
DEPRECIACOES_AMORTIZACOES['Total'] = -DEPRECIACOES_AMORTIZACOES['Total'].round(2)

In [43]:
###   (=)
RESULTADO_FINANCEIRO = df[(df['Categoria_Num'].str.startswith('0601')) | (df['Categoria_Num'].str.startswith('0602'))].copy()

RESULTADO_FINANCEIRO_0601 = RESULTADO_FINANCEIRO[RESULTADO_FINANCEIRO['Categoria_Num'].str.startswith('0601')].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
RESULTADO_FINANCEIRO_0601.columns = ['Categoria_Num', 'Categoria', 'Total']
RESULTADO_FINANCEIRO_0601['Total'] = RESULTADO_FINANCEIRO_0601['Total'].round(2)

RESULTADO_FINANCEIRO_0602 = RESULTADO_FINANCEIRO[RESULTADO_FINANCEIRO['Categoria_Num'].str.startswith('0602')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
RESULTADO_FINANCEIRO_0602.columns = ['Categoria_Num', 'Categoria', 'Total']
RESULTADO_FINANCEIRO_0602['Total'] = RESULTADO_FINANCEIRO_0602['Total'].round(2)

RESULTADO_FINANCEIRO = RESULTADO_FINANCEIRO_0601 + RESULTADO_FINANCEIRO_0602

In [44]:
###   (=)
RESULTADO_N_OPERACIONAL = df[(df['Categoria_Num'].str.startswith('0701')) | (df['Categoria_Num'].str.startswith('0702'))].copy()

RESULTADO_N_OPERACIONAL_0701 = RESULTADO_N_OPERACIONAL[RESULTADO_N_OPERACIONAL['Categoria_Num'].str.startswith('0701')].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
RESULTADO_N_OPERACIONAL_0701.columns = ['Categoria_Num', 'Categoria', 'Total']
RESULTADO_N_OPERACIONAL_0701['Total'] = RESULTADO_N_OPERACIONAL_0701['Total'].round(2)

RESULTADO_N_OPERACIONAL_0702 = RESULTADO_N_OPERACIONAL[RESULTADO_N_OPERACIONAL['Categoria_Num'].str.startswith('0702')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
RESULTADO_N_OPERACIONAL_0702.columns = ['Categoria_Num', 'Categoria', 'Total']
RESULTADO_N_OPERACIONAL_0702['Total'] = RESULTADO_N_OPERACIONAL_0702['Total'].round(2)

RESULTADO_N_OPERACIONAL = pd.concat([RESULTADO_N_OPERACIONAL_0701, RESULTADO_N_OPERACIONAL_0702])

In [45]:
###   (-)
IR_CSLL = df[df['Categoria_Num'].str.startswith('08')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
IR_CSLL.columns = ['Categoria_Num', 'Categoria', 'Total']
IR_CSLL['Total'] = IR_CSLL['Total'].round(2)

In [46]:
###   (-)
DISTRIBUICAO_LUCROS = df[df['Categoria_Num'].str.startswith('09')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
DISTRIBUICAO_LUCROS.columns = ['Categoria_Num', 'Categoria', 'Total']
DISTRIBUICAO_LUCROS['Total'] = DISTRIBUICAO_LUCROS['Total'].round(2)

In [47]:
###   (-)
INVESTIMENTO_IMOBILIZADO_INTANGIVEL = df[df['Categoria_Num'].str.startswith('10')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
INVESTIMENTO_IMOBILIZADO_INTANGIVEL.columns = ['Categoria_Num', 'Categoria', 'Total']
INVESTIMENTO_IMOBILIZADO_INTANGIVEL['Total'] = INVESTIMENTO_IMOBILIZADO_INTANGIVEL['Total'].round(2)

In [48]:
###   (+)
ENTRADA_EMPRESTIMO = df[(df['Categoria_Num'].str.startswith('11'))].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
ENTRADA_EMPRESTIMO.columns = ['Categoria_Num', 'Categoria', 'Total']
ENTRADA_EMPRESTIMO['Total'] = ENTRADA_EMPRESTIMO['Total'].round(2)

In [49]:
###   (-)
SAIDA_EMPRESTIMOS = df[df['Categoria_Num'].str.startswith('12')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
SAIDA_EMPRESTIMOS.columns = ['Categoria_Num', 'Categoria', 'Total']
SAIDA_EMPRESTIMOS['Total'] = SAIDA_EMPRESTIMOS['Total'].round(2)

In [50]:
###   (+)
APORTES_CAPITAL = df[(df['Categoria_Num'].str.startswith('13'))].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
APORTES_CAPITAL.columns = ['Categoria_Num', 'Categoria', 'Total']
APORTES_CAPITAL['Total'] = APORTES_CAPITAL['Total'].round(2)

In [51]:
OUTRAS_MOV_CAIXA = df[(df['Categoria_Num'].str.startswith('1401')) | (df['Categoria_Num'].str.startswith('1402'))].copy()

OUTRAS_MOV_CAIXA_1401 = OUTRAS_MOV_CAIXA[(OUTRAS_MOV_CAIXA['Categoria_Num'].str.startswith('1401')) & (OUTRAS_MOV_CAIXA['Categoria_Num'] != '140103')].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
OUTRAS_MOV_CAIXA_1401.columns = ['Categoria_Num', 'Categoria', 'Total']
OUTRAS_MOV_CAIXA_1401['Total'] = OUTRAS_MOV_CAIXA_1401['Total'].round(2)

OUTRAS_MOV_CAIXA_1402 = OUTRAS_MOV_CAIXA[OUTRAS_MOV_CAIXA['Categoria_Num'].str.startswith('1402')].groupby(['Categoria_Num', 'Categoria'])['Valor pago'].sum().reset_index()
OUTRAS_MOV_CAIXA_1402.columns = ['Categoria_Num', 'Categoria', 'Total']
OUTRAS_MOV_CAIXA_1402['Total'] = OUTRAS_MOV_CAIXA_1402['Total'].round(2)

OUTRAS_MOV_CAIXA = pd.concat([OUTRAS_MOV_CAIXA_1401, OUTRAS_MOV_CAIXA_1402]).dropna()


In [52]:
#RECEITA_BRUTA.sum()
RECEITA_BRUTA.sum()

Categoria_Num    0101010101010102010101050101010701010108010101...
Categoria         EB1 - ASSESSORIA NO BRASIL EB2 - NIW - ASSESS...
Total                                                  60255156.42
dtype: object

In [53]:
#RECEITA_BRASIL = df[df['Categoria_Num'].str.startswith('0101')].groupby(['Categoria_Num','Categoria'])['Valor recebido'].sum().reset_index()
#RECEITA_BRASIL.columns = ['Categoria_Num', 'Categoria', 'Total']
#RECEITA_BRASIL['Total'] = RECEITA_BRASIL['Total'].round(2)

In [54]:
#RECEITA_BRASIL1 = RECEITA_BRASIL.groupby(['Categoria_Num', 'Categoria'])['Total'].sum().reset_index()

In [55]:
#RECEITA_BRASIL

#Secundários

In [56]:
PLANODECONTAS1 = pd.concat([RECEITA_BRUTA, DEDUCAO_IMPOSTOS, CUSTOS_SERV_VENDIDO, DESPESAS_OPERACIONAIS, DEPRECIACOES_AMORTIZACOES, RESULTADO_FINANCEIRO, RESULTADO_N_OPERACIONAL, IR_CSLL, DISTRIBUICAO_LUCROS, INVESTIMENTO_IMOBILIZADO_INTANGIVEL, ENTRADA_EMPRESTIMO, SAIDA_EMPRESTIMOS, APORTES_CAPITAL, OUTRAS_MOV_CAIXA], ignore_index=True)

In [57]:
df_pc1 = PLANODECONTAS1.groupby('Categoria_Num')['Total'].sum().reset_index()
df_pc1['Categoria_Num'] = df_pc1['Categoria_Num'].replace('01', '01 ')

In [58]:
df_pc['Total'] = df_pc['Categoria_Num'].map(df_pc1.set_index('Categoria_Num')['Total']).fillna(0)

In [59]:
#df_pc = df_pc.dropna(subset=['Categoria_Num', 'Categoria'], how='all')

In [60]:
df_pc.loc[df_pc['Categoria'] == 'Transferência de Entrada', 'Categoria_Num'] = '01 '
df_pc.loc[df_pc['Categoria'] == 'Transferência de Saída', 'Categoria_Num'] = '02'
df_pc.loc[df_pc['Categoria'] == 'Total de Transferências', 'Categoria_Num'] = '0'

In [61]:
#essa aqui é a alterada para preencher
#df_pc1 = df[(df['Categoria_Num'].str.startswith('01')) | (df['Categoria_Num'] == '140103')].groupby(['Categoria_Num', 'Categoria'])['Valor recebido'].sum().reset_index()
#df_pc1.columns = ['Categoria_Num', 'Categoria', 'Total']
#df_pc1['Total'] = df_pc1['Total'].round(2)

In [62]:
#total_categoria_01 = df_pc1[df_pc1['Categoria_Num'].str.startswith('01')]['Total'].sum()
#df_pc1.loc[df_pc1['Categoria_Num'] == '01', 'Total'] = total_categoria_01

In [63]:
df_pc1.head(100)

,Categoria_Num,Total
0,01010101,106399.03
1,01010102,4405618.86
2,01010105,13019.85
3,01010107,22942.24
4,01010108,2000.00
...,...,...
95,03010102,-153618.43
96,03010103,0.00
97,0301010801,-130310.22
98,030101080202,-32040.54


In [64]:
df_pc1.head(200)

,Categoria_Num,Total
0,01010101,106399.03
1,01010102,4405618.86
2,01010105,13019.85
3,01010107,22942.24
4,01010108,2000.00
...,...,...
195,040201,-10.00
196,040202,-58.20
197,040203,-12330.77
198,040205,-16152.29


In [65]:
#df.loc[df['Categoria_Num'] == '140103']

In [66]:
with pd.ExcelWriter('C:\\Users\\Kkk\\OneDrive\\Documentos\\DNC\\Projetos\\arquivo121.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Basededados', index=False)
    df_pc1.to_excel(writer, sheet_name='Plano de contas', index=False)

In [67]:
#from google.colab import drive
#drive.mount('/content/drive')

In [68]:
#df_pc.to_excel('Dimensao.xlsx', index=False)